# Praca inżynierska
Marcin Bobiński
nr albumu: 297225

In [ ]:
import os
HOME_PATH = os.getcwd()
RESEARCH_PATH = os.path.join(HOME_PATH, "models/research")
DATASET_PATH = os.path.join(HOME_PATH, "Dataset")
PRETRAINED_MODELS_PATH = os.path.join(HOME_PATH, "pretrained_models")
DATA_PATH = os.path.join(HOME_PATH, "data")
TF_RECORD_PATH = os.path.join(DATA_PATH, "tf_record")
MY_MODEL = os.path.join(HOME_PATH, "my_model")

# Uczenie modelu
## Przygotowanie plików tf record

Config

In [ ]:
import os
import shutil

os.chdir(HOME_PATH)
if not os.path.exists(MY_MODEL):
    os.makedirs(MY_MODEL)

_ = shutil.copy(
    os.path.join(PRETRAINED_MODELS_PATH,"ssd640","pipeline.config"),
    MY_MODEL
)

Uczenie

In [ ]:
training = f"python object_detection/model_main_tf2.py \
--pipeline_config_path={os.path.join(MY_MODEL,'pipeline.config')} \
--model_dir={os.path.join(MY_MODEL,'checkpoints')} \
--alsologtostderr"
print(training)

In [ ]:
eval = f"python object_detection/model_main_tf2.py \
    --pipeline_config_path={os.path.join(MY_MODEL,'pipeline.config')} \
    --model_dir={os.path.join(MY_MODEL,'checkpoints')} \
    --checkpoint_dir={os.path.join(MY_MODEL,'checkpoints')} \
    --alsologtostderr"
print(eval)


In [ ]:
tensorboard = f"tensorboard --logdir={os.path.join(MY_MODEL,'checkpoints')}"
print(tensorboard)

Export the inference graph

In [ ]:
os.chdir(RESEARCH_PATH)

os.system(f"python object_detection/exporter_main_v2.py \
    --input_type image_tensor \
    --pipeline_config_path {os.path.join(MY_MODEL,'pipeline.config')} \
    --trained_checkpoint_dir {os.path.join(MY_MODEL,'checkpoints')} \
    --output_directory {os.path.join(MY_MODEL, 'inference_graph')}")

Export TF Lite graph

In [ ]:
os.chdir(RESEARCH_PATH)

os.system(f"python object_detection/export_tflite_graph_tf2.py \
    --pipeline_config_path {os.path.join(MY_MODEL,'pipeline.config')} \
    --trained_checkpoint_dir {os.path.join(MY_MODEL,'checkpoints')} \
    --output_directory {os.path.join(MY_MODEL, 'inference_tflite_graph')} \
    --max_detections 100 \
    --ssd_use_regular_nms False \
    "
)

os.chdir(HOME_PATH)

In [ ]:
os.chdir(RESEARCH_PATH)

os.system(f"python object_detection/export_tflite_ssd_graph.py \
    --pipeline_config_path {os.path.join(MY_MODEL,'pipeline.config')} \
    --trained_checkpoint_prefix {os.path.join(MY_MODEL,'checkpoints','ckpt-21.index')} \
    --output_directory {os.path.join(MY_MODEL, 'inference_tflite_graph')} \
    --add_postprocessing_op True \
")

os.chdir(HOME_PATH)


In [ ]:
# DEFAULT TFLITE
import tensorflow as tf

os.chdir(os.path.join(MY_MODEL, 'inference_tflite_graph'))

# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(os.path.join(MY_MODEL, 'inference_tflite_graph', 'saved_model'))

converter.optimizations = [ tf.lite.Optimize.DEFAULT ]

converter.experimental_new_converter = True
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
    tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]

tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

os.chdir(HOME_PATH)

In [ ]:
#TFLITE QUANTIZATION INT8
import tensorflow as tf

from PIL import Image
import numpy as np

dataset_path = os.path.join(DATASET_PATH, "test", "Images", "05june05_static_street_boston")

def representative_dataset():
    for image_path in os.listdir(dataset_path):
        image = np.asarray(Image.open(os.path.join(dataset_path, image_path)).convert('RGB').resize((640,640)), dtype=np.float32)
        image = np.array(np.expand_dims(image, 0), dtype=np.float32)
        yield [image]

os.chdir(os.path.join(MY_MODEL, 'inference_tflite_graph'))

# Convert the model\n",
converter = tf.lite.TFLiteConverter.from_saved_model(os.path.join(MY_MODEL, 'inference_tflite_graph', 'saved_model'))

converter.allow_custom_ops = True
converter.experimental_new_converter = True

converter.optimizations = [ tf.lite.Optimize.DEFAULT ]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8, tf.lite.OpsSet.TFLITE_BUILTINS]

converter.inference_input_type = tf.uint8
converter.representative_dataset = representative_dataset

tflite_model = converter.convert()

# Save the model.\n",
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

os.chdir(HOME_PATH)

In [ ]:
# TFLITE FLOAT 16
import tensorflow as tf

os.chdir(os.path.join(MY_MODEL, 'inference_tflite_graph'))

# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(os.path.join(MY_MODEL, 'inference_tflite_graph', 'saved_model'))

converter.optimizations = [ tf.lite.Optimize.DEFAULT ]
converter.target_spec.supported_types = [tf.float16]

converter.experimental_new_converter = True
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
    tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]

tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

os.chdir(HOME_PATH)

Mask-RCNN

In [ ]:
import tensorflow as tf
model = tf.saved_model.load(os.path.join(MY_MODEL,"inference_graph","saved_model"))

os.chdir(os.path.join(MY_MODEL, 'inference_tflite_graph'))

keras_model = model.keras_model
converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
converter.allow_custom_ops = True
converter.experimental_new_converter = True
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
    tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]

converter.optimizations = [ tf.lite.Optimize.DEFAULT ]

tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

os.chdir(HOME_PATH)

In [ ]:
import tensorflow as tf

os.chdir(os.path.join(MY_MODEL, 'inference_tflite_graph'))

converter = tf.lite.TFLiteConverter.from_saved_model(os.path.join(MY_MODEL,"inference_graph","saved_model"))
converter.allow_custom_ops = True
converter.experimental_new_converter = True
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
    tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]

# converter.optimizations = [ tf.lite.Optimize.DEFAULT ]
converter.optimizations = [ tf.lite.Optimize.OPTIMIZE_FOR_SIZE ]
# converter.optimizations = [ tf.lite.Optimize.EXPERIMENTAL_SPARSITY ]

tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

os.chdir(HOME_PATH)

Testing

In [ ]:
import tensorflow as tf

interpreter = tf.lite.Interpreter(model_path=os.path.join(MY_MODEL, "inference_tflite_graph","model.tflite"), num_threads=6)
interpreter.allocate_tensors()

# Print input shape and type
print(interpreter.get_input_details()[0]['shape'], interpreter.get_input_details()[0]['dtype'])
# Print output shape and type
for i in range(4):
    print(interpreter.get_output_details()[i]['shape'], interpreter.get_output_details()[i]['dtype'])

In [ ]:
import cv2

import timeit
import time

import tensorflow as tf

from PIL import Image
from IPython.display import display

import numpy as np

from models.research.object_detection.utils import label_map_util
from models.research.object_detection.utils import visualization_utils as vis_util

category_index = label_map_util.create_category_index_from_labelmap(os.path.join(DATA_PATH, "annotations.pbtxt"), use_display_name=True)

input_index = interpreter.get_input_details()[0]["index"]
scores_index = interpreter.get_output_details()[0]['index']
boxes_index = interpreter.get_output_details()[1]['index']
detection_num_index = interpreter.get_output_details()[2]['index']
classes_index = interpreter.get_output_details()[3]['index']

np.set_printoptions(suppress=True, precision=3)

def run_inference_and_display(image_path):
    image_np = Image.open(image_path)
    image_np = np.asarray(image_np.convert('RGB').resize((640,640)))

    image_norm = (image_np.astype(np.float32) - 127.5) / 127.5

    input_tensor = np.array(np.expand_dims(image_norm,0), dtype=np.float32)

    interpreter.set_tensor(input_index, input_tensor)

    t = time.time()
    interpreter.invoke()
    t = time.time() - t

    scores = np.squeeze(interpreter.get_tensor(scores_index))
    boxes = np.squeeze(interpreter.get_tensor(boxes_index))
    classes = np.squeeze(interpreter.get_tensor(classes_index)).astype(np.int64) + 1

    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        boxes,
        classes,
        scores,
        category_index,
        min_score_thresh=.6,
        use_normalized_coordinates=True,
        line_thickness=4)

    display(Image.fromarray(image_np))
    print("inference time:", round(t*1000), "ms")
    print("Scores:", scores * 100)
    print("Scores:", scores.sum() )

In [ ]:
EXAMPLE_IMAGES_DATA_PATH = os.path.join(HOME_PATH, "example_images")
images = list(filter(lambda x: str(x).endswith(".jpg"),os.listdir(EXAMPLE_IMAGES_DATA_PATH)))
for image in images:
    run_inference_and_display(os.path.join(EXAMPLE_IMAGES_DATA_PATH, image))
    pass

In [ ]:
import numpy as np
import os
import tensorflow as tf

from PIL import Image
from IPython.display import display

from models.research.object_detection.utils import ops as utils_ops
from models.research.object_detection.utils import label_map_util
from models.research.object_detection.utils import visualization_utils as vis_util


utils_ops.tf = tf.compat.v1
tf.gfile = tf.io.gfile

model = tf.saved_model.load(os.path.join(MY_MODEL,"inference_graph","saved_model"))
category_index = label_map_util.create_category_index_from_labelmap(os.path.join(DATA_PATH, "annotations.pbtxt"), use_display_name=True)

def run_inference_for_single_image(model, image):
    image = np.asarray(image)
    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis, ...]
    # Run inference
    model_fn = model.signatures['serving_default']
    output_dict = model_fn(input_tensor)
    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(output_dict.pop('num_detections'))
    # need_detection_key = ['detection_classes','detection_boxes','detection_masks','detection_scores']
    need_detection_key = ['detection_classes','detection_boxes','detection_scores']
    output_dict = {key: output_dict[key][0, :num_detections].numpy()
               for key in need_detection_key}
    output_dict['num_detections'] = num_detections
    # detection_classes should be ints.
    output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
    # Handle models with masks:
    if 'detection_masks' in output_dict:
        # Reframe the the bbox mask to the image size.
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            tf.convert_to_tensor(output_dict['detection_masks']), output_dict['detection_boxes'],
            image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
        output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    return output_dict

def show_inference(model, image_path):
    image_np = np.array(Image.open(image_path))
    # Actual detection.
    output_dict = run_inference_for_single_image(model, image_np)
    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        output_dict['detection_boxes'],
        output_dict['detection_classes'],
        output_dict['detection_scores'],
        category_index,
        instance_masks=output_dict.get('detection_masks_reframed', None),
        use_normalized_coordinates=True,
        line_thickness=4)

    display(Image.fromarray(image_np))

In [ ]:
EXAMPLE_IMAGES_DATA_PATH = os.path.join(HOME_PATH, "example_images")
images = list(filter(lambda x: str(x).endswith(".jpg"),os.listdir(EXAMPLE_IMAGES_DATA_PATH)))
for image in images:
   show_inference(model, os.path.join(EXAMPLE_IMAGES_DATA_PATH, image))